In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow.contrib.eager as tfe
import tensorflow as tf
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, LabelBinarizer, StandardScaler
from sklearn.decomposition import PCA

import numpy as np

/home/ubuntu/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.enable_eager_execution()

#### Define neural network

In [3]:
from annsa.model_classes import cnn_model_features,cnn,save_model
from annsa import read_spectrum

In [ ]:
scaler=make_pipeline(FunctionTransformer(np.log1p, validate=False))
mlb=LabelBinarizer()

training_data = np.load('FINAL_template_training_data.npy')
training_keys = np.load('FINAL_template_training_keys.npy')
training_keys_binarized = mlb.fit_transform(training_keys.reshape([training_data.shape[0],1]))

testing_data = np.load('FINAL_template_hyperparameter_training_data.npy')
testing_keys = np.load('FINAL_template_hyperparameter_training_keys.npy')
testing_keys_binarized = mlb.transform(testing_keys.reshape([testing_data.shape[0],1]))

### Train and save model

In [ ]:
# Using model features from previous work
model_features = cnn_model_features(learining_rate=1e-3 ,
                                      l2_regularization_scale=0.0,
                                      dropout_probability=0.45,
                                      batch_size=2**10,
                                      number_filters=(11, 9),
                                      kernel_size=(10, 50),
                                      nodes_layer_1=128,
                                      nodes_layer_2=-1,
                                      scaler=scaler)

num_epochs=19
model_features.scaler.fit(training_data)
X_tensor = tf.constant(training_data)
y_tensor = tf.constant(training_keys_binarized)
train_dataset_tensor = tf.data.Dataset.from_tensor_slices((X_tensor, y_tensor))
test_dataset = (testing_data, testing_keys_binarized)

tf.reset_default_graph()
optimizer = tf.train.AdamOptimizer(model_features.learining_rate)
model = cnn(model_features)
all_loss_train, all_loss_test = model.fit_batch(train_dataset_tensor,
                                                test_dataset,
                                                optimizer,
                                                num_epochs,
                                                early_stopping_patience=28,
                                                verbose=1,
                                                print_errors=True,
                                                max_time=3600)



In [ ]:
plt.plot(all_loss_train,label='Training loss')
plt.plot(np.linspace(0,len(all_loss_train),24),all_loss_test,label='Testing loss')
plt.ylim([0,5])
plt.legend()


In [ ]:
save_model('final-models','final_model_cnn_19epochs_v2',model,model_features)